In [1]:
from llm_study import *

In [2]:
from langchain.chains.summarize import load_summarize_chain
from langchain.llms.openai import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

### Load globally used LLMs and chains

In [10]:
doc_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

react_llm = OpenAI(model="text-davinci-003", temperature=0.5)

summarize_llm = OpenAI(model="text-davinci-003", temperature=0.0)

knowledge_summarizer = KnowledgeSummarizer(llm=summarize_llm)

chat_summarizer = ChatStateSummarizer(llm=summarize_llm)

def vectorstore_factory(*docs: Document):
    return FAISS.from_documents(docs or [KnowledgeDocTypes.blank()], doc_embeddings)

### Define ReAct examples

In [4]:
react_examples = [
    """
    Summary: You are shopping at the supermarket. A stranger approached you.
    Obs: The stranger says: So, do you like soccer?
    Act: [Think] I must recall if I like soccer
    Obs: OK
    Act: [Recall] Do I like soccer?
    Obs: Played soccer with the coworkers last sunday afternoon
    Act: [Say] Yes, I play it often with my coworkers
    """,
    """
    Summary: A man named Mark Lewis approached you making small talk about the bad weather
    Obs: Mark Lewis says: So, are you married?
    Act: [Think] I must recall if I am married
    Obs: OK
    Act: [Recall] Am I married?
    Obs: John black married Henrietta Black in 1892
    Act: [Say] Yes, since 1892. Why do you ask?
    """,
]

example_selector = ReActUtils.create_similarity_examples_selector(
    examples=[ReActExample(content=e) for e in react_examples],
    embeddings=doc_embeddings,
    vectorstore_cls=FAISS,
    num_examples=1,
)

### Define the NpcSheet model

In [ ]:
TMPL_NPC_SUMMARY = textwrap.dedent("""
    Your name is {name}
    You are {age} years old
    Your personality is {personality_traits}
    About you: {description}
""").strip()

class NpcSheet(BaseNpcSheet):
    name: str
    age: int
    personality_traits: List[str]
    description: str

    @property
    def summary(self) -> str:
        kwargs = self.__dict__
        kwargs["personality_traits"] = ", ".join(kwargs["personality_traits"])
        return TMPL_NPC_SUMMARY.format(**kwargs)

### Declare NPCs

In [25]:
john = Npc(
    sheet=NpcSheet(
        name="John Bennet",
        age=24,
        personality_traits=["polite", "gentle", "humble"],
        description="You are a lawyer",
    ),
    knowledge_base=NpcKnowledge(
        max_retrieval=2,
        min_relevance=0.7,
        summarize_chain=knowledge_summarizer,
        vectorstore=vectorstore_factory(
            KnowledgeDocTypes.chat("""
                I was enjoying a drink in a London pub when I was approached by a stranger, Buck Jones,
                who introduced himself as a dietary doctor. After I told him I did not have a sister,
                he apologized for not introducing himself sooner and we exchanged pleasantries. He then
                gave me some advice about nutrition and health, emphasizing the importance of avoiding
                sugar and carbs. I told him I was familiar with the basics of nutrition and health and
                that I try to maintain a balanced diet
            """),
        ),
    ),
)

becky = Npc(
    sheet=NpcSheet(
        name="Becky Jameson",
        age=31,
        personality_traits=["bold", "outgoing", "inquisitive"],
        description="You are a detective",
    ),
    knowledge_base=NpcKnowledge(
        max_retrieval=2,
        min_relevance=0.7,
        summarize_chain=knowledge_summarizer,
        vectorstore=vectorstore_factory(),
    ),
)

SyntaxError: invalid syntax (1319777964.py, line 16)

### Create Chats

In [11]:
def create_chat(npc: Npc, situation: str, trigger: str) -> NpcChat:
    npc.situation = situation
    tools = ReActUtils.create_tools(npc)
    agent = NpcChatAgent(tools=tools, llm=react_llm, example_selector=example_selector)
    return NpcChat(john, agent, chat_state_summarizer, trigger=trigger)

chat_1 = create_chat(john, situation="You are drinking in a pub in London", trigger="A stranger approached you")

In [12]:
chat_1.say("stranger", "Hi, do you have a sister?")



> Entering new AgentExecutor chain...

{'tool': 'Think', 'tool_input': 'I must recall if I have a sister.'}
OK
{'tool': 'Recall', 'tool_input': 'Do I have a sister?'}
 No information provided.Act: [Think] I must recall if I have a sister.
Obs: OK
Act: [Recall] Do I have a sister?
Obs: No information provided.


> Finished chain.


"No, I don't have a sister. Why do you ask?"

In [14]:
chat_1.say("stranger", "I'm sorry, I forgot to introduce myself. I'm Buck Jones, a dietary doctor")



> Entering new AgentExecutor chain...

{'tool': 'Think', 'tool_input': 'I should introduce myself to this stranger.'}
OKAct: [Think] I should introduce myself to this stranger.
Obs: OK


> Finished chain.


"Nice to meet you, Buck. I'm John Bennet."

In [15]:
chat_1.summary

'\n\nI was enjoying a drink in a London pub when a stranger, Buck Jones, introduced himself as a dietary doctor and asked if I had a sister. When I told him I did not, he apologized for not introducing himself sooner. We exchanged pleasantries and I introduced myself as John Bennet.'

In [16]:
chat_1.say("Buck Jones", "you should always remember that sugar is terrible for your health. Avoid sugar and carbs at all costs")



> Entering new AgentExecutor chain...

{'tool': 'Think', 'tool_input': 'I should recall what I already know about nutrition and health.'}
OK
{'tool': 'Recall', 'tool_input': 'I know that a balanced diet is important for overall health and that sugar should be consumed in moderation.'}
 No information provided.Act: [Think] I should recall what I already know about nutrition and health.
Obs: OK
Act: [Recall] I know that a balanced diet is important for overall health and that sugar should be consumed in moderation.
Obs: No information provided.


> Finished chain.


"Yes, I'm familiar with the basics of nutrition and health. I try to maintain a balanced diet."

In [17]:
chat_1.summary

'\n\nI was enjoying a drink in a London pub when I was approached by a stranger, Buck Jones, who introduced himself as a dietary doctor. After I told him I did not have a sister, he apologized for not introducing himself sooner and we exchanged pleasantries. He then gave me some advice about nutrition and health, emphasizing the importance of avoiding sugar and carbs. I told him I was familiar with the basics of nutrition and health and that I try to maintain a balanced diet.'

In [18]:
chat_1.finish_chat()

In [21]:
chat_2 = create_chat(john, situation="You are eating cake at home", trigger="Buck Jones approached you")

In [23]:
chat_2.say("Buck Jones", "Hi John")



> Entering new AgentExecutor chain...



> Finished chain.


'Hi, Buck. How are you?'

In [24]:
chat_2.say("Buck Jones", "What are you doing? Did I not warn you about that already?")



> Entering new AgentExecutor chain...

{'tool': 'Think', 'tool_input': 'I should recall what Buck warned me about.'}
OK
{'tool': 'Recall', 'tool_input': 'What did Buck warn me about?'}
 A stranger, Buck Jones, approached a person in a London pub and introduced himself as a dietary doctor. He gave advice about nutrition and health, emphasizing the importance of avoiding sugar and carbs. The person was familiar with the basics of nutrition and health and said they try to maintain a balanced diet.Act: [Think] I should recall what Buck warned me about.
Obs: OK
Act: [Recall] What did Buck warn me about?
Obs: A stranger, Buck Jones, approached a person in a London pub and introduced himself as a dietary doctor. He gave advice about nutrition and health, emphasizing the importance of avoiding sugar and carbs. The person was familiar with the basics of nutrition and health and said they try to maintain a balanced diet.


> Finished chain.


"Yes, you did warn me about avoiding sugar and carbs. I'm eating cake as a treat though."